In [ ]:
# !pip install rank_bm25

In [ ]:
# 라이브러리 임포트
import json
import numpy as np
import pandas as pd
from rank_bm25 import BM25Okapi

In [ ]:
# 데이터 로드
train = json.load(open('##### train.json 경로 #####', 'r', encoding='utf8')) 
test_data = json.load(open('##### test_data.json 경로 #####', 'r', encoding='utf8'))
test_question = pd.read_csv('##### test_questions.csv 경로 #####', encoding='utf8')
sample = pd.read_csv('##### sample_submission.csv 경로 #####', encoding='utf8')

# bm25 Train Set 성능 확인

### DATA

In [ ]:
number_of_contents = len(train['data'])
bound = number_of_contents

contexts = []
questions = []

for i in range(bound):
    for j in range(len(train['data'][i]['paragraphs'])):
        if len(train['data'][i]['paragraphs'][j]['qas'])>0:
            contexts.append(train['data'][i]['paragraphs'][j]['context'])
            questions.append(train['data'][i]['paragraphs'][j]['qas'][0]['question'])

In [ ]:
contexts[:3]

In [ ]:
questions[:3]

### Model

In [ ]:
bm25 = BM25Okapi([doc.split(' ') for doc in contexts])

In [ ]:
answer = []
for q in [question.split(' ') for question in questions]:
    answer.append(bm25.get_top_n(q, contexts, n=10))

In [ ]:
accumulated_RR = 0

for index, value in enumerate(answer):
    for index_, value_ in enumerate(answer[index]):
        if value_ == contexts[index]:
            accumulated_RR += 1/(1 + index_)

MRR = accumulated_RR/(len(answer))
MRR

# bm25 Test Set 추론

### DATA

In [ ]:
questions = list(test_question['question_text'])

contexts = []
par_ids = []

for i in range(len(test_data['data'])):
    for j in range(len(test_data['data'][i]['paragraphs'])):
        contexts.append(test_data['data'][i]['paragraphs'][j]['context'])
        par_ids.append(test_data['data'][i]['paragraphs'][j]['paragraph_id'])

In [ ]:
questions[:3]

In [ ]:
contexts[:3]

In [ ]:
par_ids[:3]

### Model

In [ ]:
bm25 = BM25Okapi([doc.split(' ') for doc in contexts])

In [ ]:
answer = []
for q in [question.split(' ') for question in questions]:
    answer.append(bm25.get_top_n(q, contexts, n=10))

In [ ]:
for i, a in enumerate(answer):
    for j, b in enumerate(a):
        for k, c in enumerate(contexts):
            if b == c:
                if j == 0:
                    prediction = par_ids[k]
                elif j < 9:
                    prediction = prediction + ',' + par_ids[k]
                else:
                    prediction = prediction + ',' + par_ids[k]
                    sample.loc[i, 'paragraph_id'] = prediction
sample

In [ ]:
sample.to_csv('##### prediction.csv 경로 #####', index=False, encoding="utf-8")